# An introduction to solving biological problems with Python

## Week 4: Use Cases/Practice


### Exercise 4.1.1  Filtering TSV Files

Work in pairs.

Use Case: You need to filter a large file of variants that's too big to open in Excel or R.

Write a script that reads in the variant data/test1_example1.tsv file and writes out just the Passed variants that have quality >= 50 to a new tab-separated file. 
Hint: use the Filter and Qual columns

### Use Case: Combining Multiple Files

You need to combine variants from a set of tabular patient files into a single file. You only want variants that have a variant allele frequency (vaf) > 10. But an additional complication is that you want to include variants in the PIK3CA gene that have a vaf > 5 and variants in the ESR1 gene with vaf > 2. The script below shows how the multiple patient files can be combined applying these different filter thresholds.

In [ ]:
# Load the libraries that we need
from glob import glob
from csv import DictReader

patientIDs = ["01", "10", "20"]
dirpath = "data/patients/variants/"

# Loop through the patient IDs
for id in patientIDs:
    # Generate the search path
    # Note that "*" stand for a wildcard (AKA anything)
    path = "".join([dirpath, "*", id, ".csv"])
#     print(path) # The "search" term we are using for glob
    filespath = glob(path)
#     print(filespath) # Prints the full path name to the files
    
    data = dict() # Initialise an empty dictionary
    
    # Loop through the files relevant to each patient
    for file in filespath:
        print("Filename:", file)
        with open(file, "r") as infile:
            fh = DictReader(infile) # fh is the file handle
            tmp = next(fh) # Grab the first line to help initialize the dictionary
            for k,v in tmp.items():
                data.update({k: [v]})
            # Iterate through the rest of the file and add in the values to the corresponding key
            for line in fh:
                for k,v in line.items():
                    data[k].append(v)
    keep = list() # Initialise an empty list for the indexes of the rows we want to keep
    for i in range(len(data["gene"])):
        if (data["gene"][i] == "PIK3CA" and float(data["vaf"][i]) > 5):
            keep.append(i)
        elif (data["gene"][i] == "ESR1" and float(data["vaf"][i]) > 2):
            keep.append(i)
        elif (float(data["vaf"][i]) > 10):
            keep.append(i)
    
    # Initialise an empty dictionary to store the results
    results = dict()
    # Note here the key is the column name and the value is a list
    for key, value in data.items(): # Iterate through each key, value
        for i in keep: # Iterate through the list of indexes that we want to keep
            if key not in results: # Check if the column name is in the dictionary, if not:
                results[key] = [value[i]] # Initialise the column name as a key and set the value to be a list of the value
            else:
                results[key].append(value[i]) # If the column name is already in the dictionary then add to the list
    
    # Write out the results dictionary to a file
    with open("".join(["patient_", id, "_allFiltered.csv"]), "w") as outfile:
        outfile.write(",".join(["patientID", "timepoint", "gene", "vaf"]) + "\n") # Write out the header
        lines = list(zip(results["patientID"], results["timepoint"], results["gene"], results["vaf"])) # See example below for how zip works
        # Write out each row to the output file
        for line in lines:
            outfile.write(",".join(list(line)) + "\n")

The zip function can be used to pair items, for example:

In [ ]:
list(zip(["a", "b", "c"], ["1", "2", "3"]))

### Exercise 4.2.1 Combining Multiple Files

Work in pairs.

Use case: You have 2000 files from TCGA with gene expression counts for patients (1 file per patient). You need to combine them into a single table of counts with a column for each patient. 

The count files are in data/patients/counts.zip (you will need to unzip the file) and the expected output is in the file data/patients/counts_output/combined_counts.tsv